# 1. Loading data in splits:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.5 MB/s eta 0:00:00


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # regex python
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

from nltk.tokenize import word_tokenize

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

(…)o/bert-base-cased/resolve/main/vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

(…)t-base-cased/resolve/main/tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

(…)bert-base-cased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
filename = '/content/drive/MyDrive/Colab Notebooks/input/emails.csv'
n = 50  # every 100th line = read 1% of the emails (total emails = 517400)
df = pd.read_csv(filename, header=0, skiprows=lambda i: i % n != 0)

# df = pd.read_csv(filename)
print("shape of the dataset:",df.shape)
df.head()

shape of the dataset: (10348, 2)


,file,message
0,allen-p/_sent_mail/139.,Message-ID: <9942513.1075855688311.JavaMail.ev...
1,allen-p/_sent_mail/184.,Message-ID: <4752513.1075855689288.JavaMail.ev...
2,allen-p/_sent_mail/229.,Message-ID: <9250296.1075855690264.JavaMail.ev...
3,allen-p/_sent_mail/274.,Message-ID: <22908480.1075855691237.JavaMail.e...
4,allen-p/_sent_mail/32.,Message-ID: <8220274.1075855379087.JavaMail.ev...


Let's see what individual messages looks like. They all have a set of pre-info containing various features and a body separated by two newline characters.

In [ ]:
 for i in range(20):
     print("################################################ EMAIL CONTENT NUMBER:",i,"############################################################################")
     print(df.message[i])

################################################ EMAIL CONTENT NUMBER: 0 ############################################################################
Message-ID: <9942513.1075855688311.JavaMail.evans@thyme>
Date: Tue, 12 Sep 2000 00:27:00 -0700 (PDT)
From: phillip.allen@enron.com
To: moshuffle@hotmail.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: moshuffle@hotmail.com
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf

http://www.hearme.com/vc2/?chnlOwnr=pallen@enron.com
################################################ EMAIL CONTENT NUMBER: 1 ############################################################################
Message-ID: <4752513.1075855689288.JavaMail.evans@thyme>
Date: Thu, 17 Aug 2000 08:53:00 -0700 (PDT)
From: phillip.allen@enron.com
To: mac.d.hargrove@rssmb.com
Subject: Re: enron close to 85
Mime-Version: 1.0
Content-Typ

# 2. Split the content and pre-info in message:

In [ ]:
# Dropping the file column:
df = df.drop(['file'], axis=1)

In [ ]:
# Splitting:
df['pre_info']= df.message.map(lambda r: r.split('\n\n', 1)[0])
df['content']= df.message.map(lambda r: r.split('\n\n', 1)[1])
df = df.drop(['message'], axis=1)
df.head()

,pre_info,content
0,Message-ID: <9942513.1075855688311.JavaMail.ev...,http://www.hearme.com/vc2/?chnlOwnr=pallen@enr...
1,Message-ID: <4752513.1075855689288.JavaMail.ev...,I checked into exercising options with Smith B...
2,Message-ID: <9250296.1075855690264.JavaMail.ev...,"Jeff,\n\n I have spoken to Brenda and everythi..."
3,Message-ID: <22908480.1075855691237.JavaMail.e...,---------------------- Forwarded by Phillip K ...
4,Message-ID: <8220274.1075855379087.JavaMail.ev...,I just spoke to the insurance company. They a...


In [ ]:
# Check the pre-info part:
print(df.pre_info[0])
# Keep the message id for indexing later on:
# df['message_id'] = df.pre_info.map(lambda r: r.split('\n')[0].split('Message-ID: ')[1])
# df = df.drop(['pre_info'], axis=1)

Message-ID: <9942513.1075855688311.JavaMail.evans@thyme>
Date: Tue, 12 Sep 2000 00:27:00 -0700 (PDT)
From: phillip.allen@enron.com
To: moshuffle@hotmail.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: moshuffle@hotmail.com
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Dec2000\Notes Folders\'sent mail
X-Origin: Allen-P
X-FileName: pallen.nsf


# 3. Dealing with *Forwarded by*:

In [ ]:
# #Investigating the content first:
# for i in range(25):
#     print("################################################ EMAIL CONTENT NUMBER:",i,"############################################################################")
#     print(df.content[i])

From the above samples it is clear that there are a whole lot of messages that have `---------------------- Forwarded by ...` in them.
Exactly how many? Let's see:

In [ ]:
df.content.str.contains('[- ]*Forwarded by').value_counts()

False    8318
True     2030
Name: content, dtype: int64

Let's deal with them first. How? Looking at the forwarded texts, they are accompanied by a `To, From and Subject` texts. We are interested in the body of the email that comes after the `Subject`.

The regex below simply says:
* `[- ]*` means zero or more number of `-` and `:space:` characters, because of wildcard `*`
* `\s` means all whitespace characters including newline and tabs
* `\S` means complimentary set of all the whitespace characters, basically all the alphanumeric and special characters
* Together `[\S\s]*` means zero or more characters (everything)
* `[\S\t ]*` means zero or more characters except for a newline

In [ ]:
# Test the deal with one sample email:
email = df.content[df.content.str.contains('[- ]*Forwarded by')].iloc[0]
print(email)
print("############################ END OF EMAIL ################################################################")
condition = '[- ]*Forwarded by[\S\s]*Subject:[\S\t ]*'
print(re.sub(condition, '', email).strip())

# Do it for all the others:
def deal_forwarded(row):
    condition = '[- ]*Forwarded by[\S\s]*Subject:[\S\t ]*'
    return re.sub(condition, '', row).strip()
df['content1'] = df.content.map(deal_forwarded)

---------------------- Forwarded by Phillip K Allen/HOU/ECT on 04/10/2000 
02:09 PM ---------------------------
   
	
	
	From:  Julie A Gomez                           04/01/2000 07:11 AM
	

To: Hunter S Shively/HOU/ECT@ECT, Phillip K Allen/HOU/ECT@ECT
cc:  
Subject: Alliance netback worksheet

Hello Men-

I have attached my worksheet in case you want to review the data while I am 
on holiday.

Thanks,

Julie  :-)



############################ END OF EMAIL ################################################################
Hello Men-

I have attached my worksheet in case you want to review the data while I am 
on holiday.

Thanks,

Julie  :-)


Has it worked successfully?

In [ ]:
print(df.content1.str.contains('[- ]*Forwarded by').value_counts())

False    10329
True        19
Name: content1, dtype: int64


Almost. So let's look at the pattern for these emails:

In [ ]:
# for email in df.content1[df.content1.str.contains('[- ]*Forwarded by')]:
#     print(email)
#     print("############################ END OF EMAIL ################################################################")

Looking at above emails, there's no clear pattern that can detected to remove from these emails. But we can just remove the `----- Forwarded by .. -----` alone.

In [ ]:
# Test for one email:
email = df.content1[df.content1.str.contains('[- ]*Forwarded by')].iloc[0]
print(email)
print("############################ END OF EMAIL ################################################################")
condition = '[- ]*Forwarded by[\S\s]*---[-]+'
print(re.sub(condition, '', email).strip())

# DO it for all the others:
def deal_forwarded_patternless(row):
    condition = '[- ]*Forwarded by[\S\s]*[-]+'
    return re.sub(condition, '', row).strip()
df['content2'] = df.content1.map(deal_forwarded_patternless)

---------------------- Forwarded by Hang Bui/HOU/ECT on 04/12/2000 02:14 PM 
---------------------------




A Report From the 2000 World Women's Liberation Conference The
 first speaker, a lady from England stood up and said,"During last years'
conference we spoke about being more assertive with our husbands.
 Well, after the conference I went home and told my husband,
 Barrington,that I would no longer cook for him and that he would have to do
it himself!
 After the first day, I saw nothing.  After the second day, I saw
nothing.But on the third day, I saw that he had cooked a wonderful roast
lamb." The crowd cheered.
 The second speaker, a lady from Russia stood up and said, "After last
years'conference I went home and told my husband, Ivan, that I would no
longer do his laundry and that he would have to do it himself.
After the first day, I saw nothing.  After the second day, I saw nothing.But
on the third day, I saw that he had done not only his own washing, but my
washing as well.

In [ ]:
print(df.content2.str.contains('[- ]Forwarded by').value_counts())

False    10348
Name: content2, dtype: int64


Sweet, we've eliminated all the Forwards!

# 4. Dealing with *Original Message*:

Investigating Content again:

In [ ]:
 for i in range(10,50):
     print("################################################ EMAIL CONTENT NUMBER:",i,"############################################################################")
     print(df.content2[i])

################################################ EMAIL CONTENT NUMBER: 10 ############################################################################
Yes. Trading reports to Whalley.  He is Lavorato's boss.
################################################ EMAIL CONTENT NUMBER: 11 ############################################################################
Jed,

 I understand you have been contacted regarding a telephone interview to 
discuss trading opportunities at  Enron.  I am sending you this message to 
schedule the interview.  Please call or email me with a time that would be 
convenient for you.  I look forward to speaking with you.

Phillip Allen
West Gas Trading
pallen@enron.com
713-853-7041
################################################ EMAIL CONTENT NUMBER: 12 ############################################################################
Executive Impact & Influence Program
* IMMEDIATE ACTION REQUIRED - Do Not Delete *

As part of the Executive Impact and Influence Program,

Looks like emails contain `-----Original Message-----` in them a lot. Exactly how many?

In [ ]:
df.content2.str.contains('[- ]Original Message').value_counts()

False    8893
True     1455
Name: content2, dtype: int64

Alright then the same thing applies here as well. Looking at the forwarded texts, they are accompanied by a `To, From and Subject` texts.
We are interested in the body of the email that comes after the `Subject`.

In [ ]:
# Test the deal with one sample email:
email = df.content2[df.content2.str.contains('[- ]Original Message')].iloc[0]
print(email)
print("############################ END OF EMAIL ################################################################")
condition = '[- ]*Original Message[\S\s]*Subject:[\S\t ]*'
print(re.sub(condition, '', email).strip())

# Do it for all emails:
def deal_originals(row):
    condition = '[- ]*Original Message[\S\s]*Subject:[\S\t ]*'
    return re.sub(condition, '', str(row)).strip()
df['content3'] = df.content2.map(deal_originals)

-----Original Message-----
From: 	Polsky, Phil  
Sent:	Wednesday, October 24, 2001 7:57 AM
To:	Grigsby, Mike
Subject:	Summary for Grigsby.xls

MIke,

Attached is the updated summary regarding expansions in the west.  There is only one change - PGT 2003 expansion.  The expansion has now dropped from 230,000 MM/d to 160,000 MM/d due to PPL dropping out.  

Let me know if you need anything else.

Phil
############################ END OF EMAIL ################################################################
MIke,

Attached is the updated summary regarding expansions in the west.  There is only one change - PGT 2003 expansion.  The expansion has now dropped from 230,000 MM/d to 160,000 MM/d due to PPL dropping out.  

Let me know if you need anything else.

Phil


Has it worked successfully?

In [ ]:
df.content3.str.contains('[- ]*Original Message').value_counts()

False    10332
True        16
Name: content3, dtype: int64

Almost. So let's look at the failures:

In [ ]:
# for email in df.content3[df.content3.str.contains('[- ]*Original Message')]:
#     print(email)
#     print("############################ END OF EMAIL ################################################################")

Looks like they contain patterns like this that end with `Sent:` or `Date:` instead of `Subject:`
```
----- Original Message -----
From: <Eric.Bass@enron.com>
To: <daphneco64@bigplanet.com>
Sent: Monday, December 18, 2000 1:41 PM
```

In [ ]:
# Test the deal with one sample email:
email = df.content3[df.content3.str.contains('[- ]*Original Message')].iloc[0]
print(email)
print("############################ END OF EMAIL ################################################################")
condition = '[- ]*Original Message[\S\s]*(Sent:[\S\t ]*|Date:[\S\t ]*)'
print(re.sub(condition, '', email).strip())

# Do it for all emails:
def deal_originals_new(row):
    condition = '[- ]*Original Message[\S\s]*(Sent:[\S\t ]*|Date:[\S\t ]*)'
    return re.sub(condition, '', str(row)).strip()
df['content4'] = df.content3.map(deal_originals_new)

Do you have the original flight information on our flight out of houston?

Thanks




"K. Bass" <daphneco64@bigplanet.com> on 12/18/2000 03:12:31 PM
Please respond to "K. Bass" <daphneco64@bigplanet.com>
To: Eric.Bass@enron.com
cc:  
Subject: Re:


Hi E,

Did you have to remind me???  Here goes:

Flt. 2765 Dec. 09 Iah-Phx 8:51AM
Flt. 6244 Dec. 09 Phx-Mtj 3:34PM

Flt. 6241 Dec. 14 Mtj-Phx 9:00AM
Flt. 2273 Dec. 14 Phx-Iah  1:15PM

Is Christmas Eve dinner and Christmas Day brunch o.k. with you?  It will
work better for Jason and Donnita.

LU-M
----- Original Message -----
From: <Eric.Bass@enron.com>
To: <daphneco64@bigplanet.com>
Sent: Monday, December 18, 2000 1:41 PM


> Hey Mom,
>
> Can you send me the flight numbers for the America West debacle we just
> experienced?
>
> Thanks
>
############################ END OF EMAIL ################################################################
Do you have the original flight information on our flight out of houston?

Thanks




"K. Bass" <daph

In [ ]:
# Check again:
print(df.content4.str.contains('[- ]*Original Message').value_counts())
for email in df.content4[df.content4.str.contains('[- ]*Original Message')]:
    print(email)
    print('############################################## END OF EMAIL ###############################################')

False    10342
True         6
Name: content4, dtype: int64
Some more info on the issue from Jeanne.



Attached is a ruling by ALJ DeUlloa issued yesterday pertaining to the
calculation of the California Procurement Adjustment.  A draft decision was
suppose to be release on such on Friday, March 16th.  It was not, and now we
know why.  On March 14th, the Department of Water Resources wrote to the
Commission to give the commission its views on how the CPA should be
calculated. In his ruling of yesterday DeUlloa stated that he would wait to
issue the PD so as to take into account DWR's submission. He also provided
parties until tomorrow to file comments on the DWR letter (which is attached
to the ruling).

Not surprising, DWR has a different view of the world than the UDCs do when
it comes to calculating the CPA.  The UDCs view the CPA as residual amount
left over from the generation component of their bundled rate on 1/5/01
after the cost of their retained generation (including QFs and 

The above few emails have varied patterns as they have different use cases of Original message. So leaving them here.

# 5. Dealing with *From*:

Checking content once again

In [ ]:
 for i in range(50,90):
     print("################################################ EMAIL CONTENT NUMBER:",i,"############################################################################")
     print(df.content4[i])

################################################ EMAIL CONTENT NUMBER: 50 ############################################################################
testing
################################################ EMAIL CONTENT NUMBER: 51 ############################################################################
Kay & Neal,

Thanks for remembering my birthday.  You beat my parents by one day.  

The family is doing fine.  Grace is really smiling.  She is a very happy baby 
as long as she is being held.

It sounds like your house is coming along fast.  I think my folks are ready 
to start building.  

We will probably visit in late June or July.  May is busy.  We are taking the 
kids to Disney for their birthdays.

Good luck on the house.

Keith
################################################ EMAIL CONTENT NUMBER: 52 ############################################################################
mary,

Are you sure you did the attachment right.  There was no file attached to 
your message.  P

From the above samples, we can see that there are quite a few entries with this pattern with the `From:` entry:
```
From: John J Lavorato/ENRON@enronXgate on 02/19/2001 10:19 AM
To: John Arnold/HOU/ECT@ECT
cc:  
Subject:
```
So let's deal with them.

In [ ]:
# emails containing the pattern:
df.content4.str.contains('From:[\S\s]*Subject:[\S \t]*').value_counts()

False    9889
True      459
Name: content4, dtype: int64

In [ ]:
# Test the deal with one sample email:
email = df.content4[df.content4.str.contains('From:[\S\s]*Subject:[\S \t]*')].iloc[0]
print(email)
print("############################ END OF EMAIL ################################################################")
condition = 'From:[\S\s]*Subject:[\S \t]*'
print(re.sub(condition, '', email).strip())

# Do it for all emails:
def deal_from(row):
    condition = 'From:[\S\s]*Subject:[\S \t]*'
    return re.sub(condition, '', str(row)).strip()
df['content5'] = df.content4.map(deal_from)

Leslie,after seeing point # 3 in writing , I would be extremely reluctant to 
submit. This kind of conjecture about  market manipulation , coming from us. 
would only serve to fuel the fires of the naysayers- I would delete. Thanks.


From: Leslie Lawner on 12/12/2000 11:56 AM
To: Christi L Nicolay/HOU/ECT@ECT, Joe Hartsoe/Corp/Enron@ENRON, Rebecca W 
Cantrell/HOU/ECT@ECT, Ruth Concannon/HOU/ECT@ECT, Stephanie 
Miller/Corp/Enron@ENRON, Phillip K Allen/HOU/ECT@ECT, Jane M 
Tholt/HOU/ECT@ECT, Richard Shapiro/NA/Enron@Enron
cc:  

Subject: Talking points about California Gas market

Here is my stab at the talking points  to be sent in to FERC along with the 
gas pricing info they requested for the California markets.  Let me or 
Christi know if you have any disagreements, additions, whatever.  I am 
supposed to be out of here at 2:15 today, so if you have stuff to add after 
that, get it to Christi.  Thanks.
############################ END OF EMAIL #######################################

Is the task complete?

In [ ]:
df.content5.str.contains('From:').value_counts()

False    10305
True        43
Name: content5, dtype: int64

Nope, so what went wrong?

In [ ]:
# for email in df.content5.loc[df.content5.str.contains('From:')]:
#     print(email)
#     print("############################ END OF EMAIL ################################################################")

Let's leave at this for now, because there are no recognizable patterns here.

# 6. Dealing with *To*:

#### Various patters to deal with here, so brace yourselves!!!
```
"Mark Sagel" <msagel@home.com> on 05/13/2001 09:23:02 PM
To: "John Arnold" <jarnold@enron.com>
cc:  
Subject: Natural gas update
```
```
Ina Rangel
05/10/2001 05:47 PM
To:	John Arnold/HOU/ECT@ECT
cc:
Subject:	Question?
```
```
Kristin Gandy@ENRON
09/11/2000 09:30 AM
To: Mark Koenig/Corp/Enron@ENRON, Kevin Garland/Enron Communications@Enron
Communications, Jonathan Davis/HOU/ECT@ECT, Jian
Miao/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Ed Wood/HOU/ECT@ECT, Jun Wang/Enron
Communications@Enron Communications, Miguel Vasquez/HOU/ECT@ECT, Lee
Jackson/HOU/ECT@ECT, Amber Hamby/Corp/Enron@Enron, Jay Hawthorn/Enron
Communications@Enron Communications, Joe Gordon/Corp/Enron@Enron, Susan
Edison/Enron Communications@Enron Communications, Vikas
Dwivedi/NA/Enron@Enron, Mark Courtney/HOU/ECT@ECT, Monica Rodriguez/Enron
Communications@enron communications
cc: Seung-Taek Oh/NA/Enron@ENRON, John Arnold/HOU/ECT@ECT, Andy
Zipper/Corp/Enron@Enron, George McClellan/HOU/ECT@ECT, David
Oxley/HOU/ECT@ECT
Subject: Vanderbilt Presentation and Golf Tournament
```
```
Date: Thu, 23 May 2002 11:03:58 -0500
Message-ID: <C4F6659E22D8194B925933CC808802540FC15D@server4.zilkha.com>
X-MS-Has-Attach:
X-MS-TNEF-Correlator:
Thread-Topic: Mill Run & Somerset Monthly reports
Thread-Index: AcICc2zvlTjbDznqRMKWdw4R6bKDng==
From: "Gary Verkleeren" <GVerkleeren@zilkha.com>
To: <kurt.anderson@ps.ge.com>
Cc: "Rick Winsor" <rwinsor@zilkha.com>, "Mark Haller" <mhaller@zilkha.com>,
<joseph.thorpe@ps.ge.com>
```

In [ ]:
# emails containing `To`:
df.content5.str.contains('To:[\S\s]*Subject:[\S\t ]*').value_counts()

False    9325
True     1023
Name: content5, dtype: int64

In [ ]:
# emails containing `To` on a new line:
df.content5.str.contains('\n[\S \t]*To:[\S\s]*Subject:[\S\t ]*').value_counts()

False    9327
True     1021
Name: content5, dtype: int64

In [ ]:
# emails containing zero or one line before `To`:
df.content5.str.contains('\n[\S \t]*\n[\S \t]*To:[\S\s]*Subject:[\S\t ]*').value_counts()

False    9329
True     1019
Name: content5, dtype: int64

In [ ]:
# emails containing zero or one or two lines before `To`:
df.content5.str.contains('\n[\S \t]*\n[\S \t]*\n[\S \t]*To:[\S\s]*Subject:[\S\t ]*').value_counts()

False    9338
True     1010
Name: content5, dtype: int64

In [ ]:
# # Investigate +_+
# for email in df.content5.loc[df.content5.str.contains('\n[\S \t]*\n[\S \t]*\n[\S \t]*To:[\S\s]*Subject:[\S\t ]*')]:
#     print(email)
#     print("############################ END OF EMAIL ################################################################")

In [ ]:
# Test the deal with one sample email:
email = df.content5.loc[df.content5.str.contains('\n[\S \t]*\n[\S \t]*\n[\S \t]*To:[\S\s]*Subject:[\S\t ]*')].iloc[0]
print(email)
print("############################ END OF EMAIL ################################################################")
condition = '\n[\S \t]*\n[\S \t]*\n[\S \t]*To:[\S\s]*Subject:[\S \t]*'
print(re.sub(condition, '', email).strip())

# Do it for all emails:
def deal_to(row):
    condition = '\n[\S \t]*\n[\S \t]*\n[\S \t]*To:[\S\s]*Subject:[\S \t]*'
    return re.sub(condition, '', str(row)).strip()
df['content6'] = df.content5.map(deal_to)

and they say it was purely coincidental the announcement came today.

6 is fine.  




"Jennifer White" <jenwhite7@zdnetonebox.com> on 12/13/2000 01:18:41 PM
To: John.Arnold@enron.com
cc:  
Subject: 


Hmmm... interesting news at Enron today.  Should I plan to come to your
place around 6PM?


___________________________________________________________________
To get your own FREE ZDNet Onebox - FREE voicemail, email, and fax,
all in one place - sign up today at http://www.zdnetonebox.com
############################ END OF EMAIL ################################################################
and they say it was purely coincidental the announcement came today.

6 is fine.  





Hmmm... interesting news at Enron today.  Should I plan to come to your
place around 6PM?


___________________________________________________________________
To get your own FREE ZDNet Onebox - FREE voicemail, email, and fax,
all in one place - sign up today at http://www.zdnetonebox.com


See if we have eliminated all `To:` to `Subject:` as well:

In [ ]:
df.content6.str.contains('To:[\S\s]*Subject:[\S\t ]*').value_counts()

False    10334
True        14
Name: content6, dtype: int64

Not yet, so what do we have here?

In [ ]:
# for email in df.content6.loc[df.content6.str.contains('To:[\S\s]*Subject:[\S\t ]*')]:
#     print(email)
#     print("############################ END OF EMAIL ################################################################")

All of these weren't detected because they don't have new line characters in the begenning. But they are easy to remove now!

In [ ]:
# Test the deal with one sample email:
email = df.content6.loc[df.content6.str.contains('To:[\S\s]*Subject:[\S\t ]*')].iloc[0]
print(email)
print("############################ END OF EMAIL ################################################################")
condition = '[\S\t ]*\nTo:[\S\s]*Subject:[\S \t]*'
print(re.sub(condition, '', email).strip())

# Do it for all emails:
def deal_to_new(row):
    condition = '[\S\t ]*\nTo:[\S\s]*Subject:[\S \t]*'
    return re.sub(condition, '', str(row)).strip()
df['content7'] = df.content6.map(deal_to_new)

Trey Hardy
04/17/2000 10:32 AM
To: Cliff Baxter/HOU/ECT@ECT, Jere C Overdyke/HOU/ECT@ECT, Andrea V 
Reed/HOU/ECT@ECT, George McClellan/HOU/ECT@ECT, Janet R Dietrich/HOU/ECT@ECT, 
John J Lavorato/Corp/Enron@Enron, Jeffrey A Shankman/HOU/ECT@ECT, Edward 
Ondarza/HOU/ECT@ECT, Brian Redmond/HOU/ECT@ECT, Max Yzaguirre/NA/Enron@ENRON, 
Christopher F Calger/PDX/ECT@ECT, James A Ajello/HOU/ECT@ECT, Sally 
Beck/HOU/ECT@ECT, Jim Fallon/HOU/ECT@ECT, Gary Hickerson/HOU/ECT@ECT, David W 
Delainey/HOU/ECT@ECT, Michael Miller/HOU/ECT@ECT, Kevin M Presto/HOU/ECT@ECT, 
W David Duran/HOU/ECT@ECT, Jeff Donahue/HOU/ECT@ECT, Raymond 
Bowen/HOU/ECT@ECT, Richard Lydecker/Corp/Enron@Enron, C John 
Thompson/Corp/Enron@ENRON, Greg Piper/Corp/Enron@Enron, Wes 
Colwell/HOU/ECT@ECT, Mark Tawney/HOU/ECT@ECT
cc:  
Subject: Management Summary 04/17
############################ END OF EMAIL ################################################################
Trey Hardy


In [ ]:
df.content7.str.contains('To:').value_counts()

False    10248
True       100
Name: content7, dtype: int64

In [ ]:
# for email in df.content7.loc[df.content7.str.contains('To:')]:
#     print(email)
#     print("############################ END OF EMAIL ################################################################")

Looking above, there's no clear pattern emerging. Hence, let's keep them in tab while we look at other problems.

# 7. The '*=20*' problem:

Honestly, this one is such a nuisance! I believe this character comes due to some read error/ incompatible character set (probably not UTF-8) that Enron was using

In [ ]:
print(df.content7.str.contains('=20').value_counts())
print(df.content7.str.contains('=20|=10|=09|=01').value_counts())
print(df.content7.str.contains('=\d\d').value_counts())

False    9941
True      407
Name: content7, dtype: int64
False    9826
True      522
Name: content7, dtype: int64
False    9665
True      683
Name: content7, dtype: int64


In [ ]:
# for email in df.content7.loc[df.content7.str.contains('=20|=10|=09|=01')]:
#     print(email)
#     print("############################ END OF EMAIL ################################################################")

In [ ]:
# Test the deal with one email:
email = df.content7.loc[df.content7.str.contains('=20|=10|=09|=01')].iloc[1]
print(email)
print("############################ END OF EMAIL ################################################################")
# condition = '[=]+[\n\t =]*\d\d'
# |[=\n]*[=10]|[=\n]*[=01]|[=\n]*[=09]'
# [\w]=[\w]'
condition1 = '[=]+\d\d'
condition2 = '[=]+[ \n]+'
email = re.sub(condition1, '', email)
print("############################## AFTER COND 1 ###############################################################")
print(re.sub(condition, '', email).strip())
email = re.sub(condition2, '', email)
print("############################## AFTER COND 2 ###############################################################")
print(re.sub(condition, '', email).strip())


# Do this for all emails:
def deal_equalsto(row):
    condition1 = '[=]+\d\d'
    condition2 = '[=]+[ \n]+'
    row = re.sub(condition1, '', str(row))
    return re.sub(condition2, '', str(row)).strip()
df['content8'] = df.content7.map(deal_equalsto)

If you cannot read this email, please click here .=20

Earnings.com - CPN Upgrade/Downgrade History
Earnings.com =09[IMAGE] =09
=09 [IMAGE] View Today's Upgrades/Downgrades/Coverage Initiated  Briefing  =
Calpine Corporation (CPN)       Date Brokerage Firm Action Details 10/23/20=
01  Freidman, Billings  Coverage Initiated at Buy    10/02/2001  Credit Lyo=
nnais  Upgraded to Add from Hold    09/28/2001  First Union Capital  Covera=
ge Initiated at Strong Buy    09/05/2001  Legg Mason  Coverage Initiated at=
 Strong Buy    07/25/2001  Credit Lyonnais  Downgraded to Hold  from Buy   =
 07/24/2001  Salomon Smith Barney  Downgraded to Outperform  from Buy    07=
/09/2001  Banc of America  Upgraded to Strong Buy from Buy    06/01/2001  B=
ear Stearns  Coverage Initiated at Neutral    03/13/2001  Lehman Brothers  =
Coverage Initiated at Strong Buy    01/19/2001  ABN AMRO  Upgraded to Buy f=
rom Add    01/19/2001  Banc of America  Coverage Initiated at Buy    12/18/=
2000  Credit Lyonnais  Co

In [ ]:
print(df.content8.str.contains('=20').value_counts())
print(df.content8.str.contains('=20|=10|=09|=01').value_counts())
# print(df.content8.str.contains('=\d\d').value_counts())

False    10348
Name: content8, dtype: int64
False    10348
Name: content8, dtype: int64


Sweet! All intended pre-procesing has been done!!! Let's take a look at the final data :)

In [ ]:
for i in range(1000, 1050):
    print("################################################ EMAIL CONTENT NUMBER:",i,"############################################################################")
    print(df.content8[i])

################################################ EMAIL CONTENT NUMBER: 1000 ############################################################################
One other thing -- Texas law will apply to those employees in Texas, as state courts will not allow other laws to govern in employee matters.  Oregon is the same.

As I re-read, I found more references to vesting or payout dates.  Again, these should be more generic or blank.

Michelle



 << File: Netco retention general 122101.doc >>
################################################ EMAIL CONTENT NUMBER: 1001 ############################################################################
It might help if I gave you the names!

Amy Flores
Michael Hinds
Chris Loehr
Michelle Stroud
Joyce Tang
Ben Thomason
Rudy de la Paz
Frank Felicia

Thanks!



Sheila,

Could you please provide me with PH files and also complete the charts on the following people?  Some of them may have left the company, and others may never have been employed.  Will you h

In [ ]:
df.head(10)

,pre_info,content,content1,content2,content3,content4,content5,content6,content7,content8
0,Message-ID: <9942513.1075855688311.JavaMail.ev...,http://www.hearme.com/vc2/?chnlOwnr=pallen@enr...,http://www.hearme.com/vc2/?chnlOwnr=pallen@enr...,http://www.hearme.com/vc2/?chnlOwnr=pallen@enr...,http://www.hearme.com/vc2/?chnlOwnr=pallen@enr...,http://www.hearme.com/vc2/?chnlOwnr=pallen@enr...,http://www.hearme.com/vc2/?chnlOwnr=pallen@enr...,http://www.hearme.com/vc2/?chnlOwnr=pallen@enr...,http://www.hearme.com/vc2/?chnlOwnr=pallen@enr...,http://www.hearme.com/vc2/?chnlOwnr=pallen@enr...
1,Message-ID: <4752513.1075855689288.JavaMail.ev...,I checked into exercising options with Smith B...,I checked into exercising options with Smith B...,I checked into exercising options with Smith B...,I checked into exercising options with Smith B...,I checked into exercising options with Smith B...,I checked into exercising options with Smith B...,I checked into exercising options with Smith B...,I checked into exercising options with Smith B...,I checked into exercising options with Smith B...
2,Message-ID: <9250296.1075855690264.JavaMail.ev...,"Jeff,\n\n I have spoken to Brenda and everythi...","Jeff,\n\n I have spoken to Brenda and everythi...","Jeff,\n\n I have spoken to Brenda and everythi...","Jeff,\n\n I have spoken to Brenda and everythi...","Jeff,\n\n I have spoken to Brenda and everythi...","Jeff,\n\n I have spoken to Brenda and everythi...","Jeff,\n\n I have spoken to Brenda and everythi...","Jeff,\n\n I have spoken to Brenda and everythi...","Jeff,\n\n I have spoken to Brenda and everythi..."
3,Message-ID: <22908480.1075855691237.JavaMail.e...,---------------------- Forwarded by Phillip K ...,Hello Men-\n\nI have attached my worksheet in ...,Hello Men-\n\nI have attached my worksheet in ...,Hello Men-\n\nI have attached my worksheet in ...,Hello Men-\n\nI have attached my worksheet in ...,Hello Men-\n\nI have attached my worksheet in ...,Hello Men-\n\nI have attached my worksheet in ...,Hello Men-\n\nI have attached my worksheet in ...,Hello Men-\n\nI have attached my worksheet in ...
4,Message-ID: <8220274.1075855379087.JavaMail.ev...,I just spoke to the insurance company. They a...,I just spoke to the insurance company. They a...,I just spoke to the insurance company. They a...,I just spoke to the insurance company. They a...,I just spoke to the insurance company. They a...,I just spoke to the insurance company. They a...,I just spoke to the insurance company. They a...,I just spoke to the insurance company. They a...,I just spoke to the insurance company. They a...
5,Message-ID: <24558320.1075855724141.JavaMail.e...,"Ray,\n\nIs there any detail on the gas cost pr...","Ray,\n\nIs there any detail on the gas cost pr...","Ray,\n\nIs there any detail on the gas cost pr...","Ray,\n\nIs there any detail on the gas cost pr...","Ray,\n\nIs there any detail on the gas cost pr...","Ray,\n\nIs there any detail on the gas cost pr...","Ray,\n\nIs there any detail on the gas cost pr...","Ray,\n\nIs there any detail on the gas cost pr...","Ray,\n\nIs there any detail on the gas cost pr..."
6,Message-ID: <24677137.1075855725220.JavaMail.e...,---------------------- Forwarded by Phillip K ...,(See attached file: F828FA00.PDF)\n\n\n(See at...,(See attached file: F828FA00.PDF)\n\n\n(See at...,(See attached file: F828FA00.PDF)\n\n\n(See at...,(See attached file: F828FA00.PDF)\n\n\n(See at...,(See attached file: F828FA00.PDF)\n\n\n(See at...,(See attached file: F828FA00.PDF)\n\n\n(See at...,(See attached file: F828FA00.PDF)\n\n\n(See at...,(See attached file: F828FA00.PDF)\n\n\n(See at...
7,Message-ID: <5319717.1075855726209.JavaMail.ev...,---------------------- Forwarded by Phillip K ...,Phillip:\n\nHere's a summary of our current fo...,Phillip:\n\nHere's a summary of our current fo...,Phillip:\n\nHere's a summary of our current fo...,Phillip:\n\nHere's a summary of our current fo...,Phillip:\n\nHere's a summary of our current fo...,Phillip:\n\nHere'

# 8. Other patterns or problems that creep in:

We find html links sometimes:

```
Planned subsidies would distort competition

Oct. 23, 2001 (Helsingin Sanomat) -- The Finnish forest industry has severely criticised the financing arrangements of the new pulp ...
( Story... <http://www.forestweb.com/digest/news.control.taf?_section=liststories&_function=detail&FORESTWEBNEWS_uid1612&_UserReference=9BACFC3BFDBDF3C13BD590B2>)


top of page <http://www.forestweb.com/digest/news.control.taf?_section=view&_function=detail&EDITION_ID96&_UserReference=9BACFC3BFDBDF3C13BD590B2#top>
  _____  
```
Or it has characters like these: `>`, which are mostly forwards from other email services like Yahoo!
```
CECECOMCOM@aol.com <mailto:CECECOMCOM@aol.com> wrote:

> ATTACHMENT part 2 message/rfc822 Date: Thu, 13 Dec 2001 11:48:47 -0800 (PST)
> FYI
>
> Roadside Assistance: Something all Texas women should know... especially
> since this has received very little publicity.
>
> Your Texas drivers license has a phone number in small print on the
> back,
> just above the bar code: 1-800-525-5555
```
But these are found once every hundred emails, hence it's safe to ignore them.

# 3. Pre-processing for NLP:

Processing for NLP requires us to tokenize and select emails that confirm to certain expectations that we set

In [ ]:
# Getting the final content8 out:
df_nlp = df[['pre_info', 'content8']]
# Removing the emails that are empty:
df_nlp = df_nlp.loc[~(df_nlp.content8=='')]
df_nlp = df_nlp.rename(columns={'content8':'content'}).reset_index(drop=True)
print(df_nlp.shape)
df_nlp.head()

(10288, 2)


,pre_info,content
0,Message-ID: <9942513.1075855688311.JavaMail.ev...,http://www.hearme.com/vc2/?chnlOwnr=pallen@enr...
1,Message-ID: <4752513.1075855689288.JavaMail.ev...,I checked into exercising options with Smith B...
2,Message-ID: <9250296.1075855690264.JavaMail.ev...,"Jeff,\n\n I have spoken to Brenda and everythi..."
3,Message-ID: <22908480.1075855691237.JavaMail.e...,Hello Men-\n\nI have attached my worksheet in ...
4,Message-ID: <8220274.1075855379087.JavaMail.ev...,I just spoke to the insurance company. They a...


In [ ]:
df_nlp.to_csv('/content/drive/MyDrive/Colab Notebooks/data/enron-pre-processed.csv')

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# # Tokenize using your fav tokenizer:
# # This step is quite time consuming:
df_nlp['bert_tokens'] = df_nlp['content'].map(lambda r: tokenizer.tokenize(r))
df_nlp['nltk_tokens'] = df_nlp['content'].map(lambda r: word_tokenize(r))

This is what you've been looking for!!

In [ ]:
df_nlp.to_csv('/content/drive/MyDrive/Colab Notebooks/data/enron-pre-processed-nlp.csv')

 Consider only emails that have more than 10 tokens and less than 512 tokens for BERT processing purposes:

In [ ]:
# Saving a file for BERT transformer models:
df_out_final = df_nlp.content.loc[df_nlp['bert_tokens'].map(lambda r: (len(r)>10 and len(r)<512))].reset_index(drop=True)
print(df_out_final.shape)
# Save the final contents:
df_out_final.to_csv('/content/drive/MyDrive/Colab Notebooks/data/enron-processed-final.csv',sep='\t')

(8477,)


In [ ]:
token_list = [token for row in df_nlp['nltk_tokens'] for token in row]
# token_list = [token for row in df_nlp['bert_tokens'] for token in row]
token_series = pd.Series(token_list)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(token_series.value_counts())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Viola! Your emails are ready for Large scale language models like BERT! Check the output directory :)